In [12]:
import pandas as pd

df = pd.read_csv(r'gen_dataset.csv')
text_data = list(df["text"])
class_data = df["class"].to_numpy()
for i in range(len(class_data)):
    if class_data[i]=='notsarc':
        class_data[i]=0
    else:
        class_data[i]=1
class_data = list(class_data)
train_sarc_text_data = text_data[:int(len(text_data) * 0.8)]
test_sarc_text_data = text_data[int(len(text_data) * 0.8):]

train_sarc_class_data = class_data[:int(len(class_data) * 0.8)]
test_sarc_class_data = class_data[int(len(class_data) * 0.8):]
df

,class,id,text
0,0,1,"If that's true, then Freedom of Speech is doom..."
1,0,2,Neener neener - is it time to go in from the p...
2,0,3,"Just like the plastic gun fear, the armour pie..."
3,0,4,So geology is a religion because we weren't he...
4,0,5,Well done Monty. Mark that up as your first ev...
...,...,...,...
6515,1,6516,depends on when the baby bird died. run alon...
6516,1,6517,"ok, sheesh, to clarify, women who arent aborti..."
6517,1,6518,so.. eh?? hows this sound? will it fly w...
6518,1,6519,"I think we should put to a vote, the right of ..."


In [13]:
df = pd.read_csv(r'irony.csv')
text_data1 = list(df["comment_text"])
class_data1 = df["label"].to_numpy()
for i in range(len(class_data1)):
    if class_data1[i]==1:
        class_data1[i]=1
    else:
        class_data1[i]=0
class_data1 = list(class_data1)    
train_irony_text_data = text_data1[:int(len(text_data1) * 0.8)]
test_irony_text_data = text_data1[int(len(text_data1) * 0.8):]

train_irony_class_data = class_data1[:int(len(class_data1) * 0.8)]
test_irony_class_data = class_data1[int(len(class_data1) * 0.8):]
df

,comment_text,label
0,I suspect atheists are projecting their desire...,0
1,It's funny how the arguments the shills are ma...,0
2,We are truly following the patterns of how the...,0
3,air pressure dropping as altitude goes higher....,0
4,Absolutely. I think we'd be hard pressed to f...,0
...,...,...
1944,This is an interesting point. There are no sh...,0
1945,Maybe you mean how we are to respond to the go...,0
1946,[... What? ](http://tinyurl. com/kw5cpxz),0
1947,Does anybody remember during one of the debate...,0


In [14]:
import torch
import torch.nn as nn
import torch.optim as optim
from transformers import BertTokenizer, BertModel
from torchmetrics.classification import BinaryAccuracy

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(device)

# Load the pre-trained BERT model and tokenizer
model = BertModel.from_pretrained('bert-base-uncased')
model.to(device)
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')

# Define the classification model
class BertClassifier(nn.Module):
    def __init__(self, bert_model, num_classes):
        super(BertClassifier, self).__init__()
        self.bert = bert_model
        self.fc = nn.Linear(768, num_classes)

    def forward(self, input_ids, attention_mask):
        outputs = self.bert(input_ids=input_ids, attention_mask=attention_mask)
        pooled_output = outputs[1]
        logits = self.fc(pooled_output)
        return logits



# Tokenize the text data
train_encodings = tokenizer(list(train_irony_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_irony_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_irony_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_irony_class_data))
)


# Define the training parameters
classifier = BertClassifier(model, num_classes=2)
classifier.to(device)
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(classifier.parameters(), lr=2e-5)
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(20):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   

cuda


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


Epoch  1
-----
Train Loss:  0.5819083896967081	Test Loss:  0.5252120637772034
Train Accuracy:  71.63003663221996% 	Test Accuracy:  73.9795918367347% 
Epoch  2
-----
Train Loss:  0.4852517970861533	Test Loss:  0.5834125700045605
Train Accuracy:  75.83333333333333% 	Test Accuracy:  66.07142857142857% 
Epoch  3
-----
Train Loss:  0.29017204646116646	Test Loss:  0.5950368761408086
Train Accuracy:  88.78205128205127% 	Test Accuracy:  67.26190478217845% 
Epoch  4
-----
Train Loss:  0.10726751703768969	Test Loss:  0.7887856425831512
Train Accuracy:  96.97802198238861% 	Test Accuracy:  69.47278915619364% 
Epoch  5
-----
Train Loss:  0.041324198300926346	Test Loss:  0.9356760869981074
Train Accuracy:  98.91025641025641% 	Test Accuracy:  74.74489795918367% 
Epoch  6
-----
Train Loss:  0.01701887592744942	Test Loss:  1.0516176320878523
Train Accuracy:  99.74358974358975% 	Test Accuracy:  71.42857142857143% 
Epoch  7
-----
Train Loss:  0.011749021230169979	Test Loss:  1.062585480359136
Train Accur

In [15]:
# Load the training data
# train_texts = ["Example sentence 1", "Example sentence 2", ...]
# train_labels = [0, 1, ...]
train_encodings = tokenizer(list(train_sarc_text_data), padding=True, truncation=True, max_length=128)
test_encodings = tokenizer(list(test_sarc_text_data), padding=True, truncation=True, max_length=128)

# Convert the data into PyTorch tensors
train_dataset = torch.utils.data.TensorDataset(
    torch.tensor(train_encodings['input_ids']),
    torch.tensor(train_encodings['attention_mask']),
    torch.tensor(list(train_sarc_class_data))
)

test_dataset = torch.utils.data.TensorDataset(
    torch.tensor(test_encodings['input_ids']),
    torch.tensor(test_encodings['attention_mask']),
    torch.tensor(list(test_sarc_class_data))
)


# Define the training parameters
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=8, shuffle=True)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=8, shuffle=True)
metric = BinaryAccuracy().to(device)


# Train the BERT classifier
classifier.train()
for epoch in range(20):
    print("Epoch ", epoch+1)
    train_losses = []
    train_acc = []
    test_losses = []
    test_acc = []
    for idx, batch in enumerate(train_loader):
        # batch.to(device)
        if(idx % 40 == 0):
            print("-", end="")
        optimizer.zero_grad()
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        loss.backward()
        train_losses.append(loss.cpu().detach().numpy())
        optimizer.step()
        op = torch.argmax(outputs, axis = 1)
        train_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
    for idx, batch in enumerate(test_loader):
        # batch.to(device)
        outputs = classifier(input_ids=batch[0].to(device), attention_mask=batch[1].to(device))
        loss = criterion(outputs, batch[2].to(device))
        test_losses.append(loss.cpu().detach().numpy())
        op = torch.argmax(outputs, axis = 1)
        test_acc.append(metric(op, batch[2].to(device)).cpu().detach().numpy())
        
        
    print()
    print("Train Loss: ",sum(train_losses) / len(train_losses), end = "\t")
    print("Test Loss: ",sum(test_losses) / len(test_losses), end = "\n")   
    print("Train Accuracy: ",(sum(train_acc) / len(train_acc)) * 100, end = "% \t")
    print("Test Accuracy: ",(sum(test_acc) / len(test_acc)) * 100, end = "% \n")   
    print()

Epoch  1
-----------------
Train Loss:  0.5084797797918869	Test Loss:  0.4812579846089603
Train Accuracy:  75.93941717791411% 	Test Accuracy:  75.76687116564416% 

Epoch  2
-----------------
Train Loss:  0.32201710111958665	Test Loss:  0.5412274413496438
Train Accuracy:  86.44555214723927% 	Test Accuracy:  77.30061349693251% 

Epoch  3
-----------------
Train Loss:  0.1373044328365555	Test Loss:  0.5910620266925887
Train Accuracy:  95.14953987730061% 	Test Accuracy:  77.99079754601227% 

Epoch  4
-----------------
Train Loss:  0.06251699322942812	Test Loss:  0.791407327626209
Train Accuracy:  97.83358895705521% 	Test Accuracy:  76.99386503067484% 

Epoch  5
-----------------
Train Loss:  0.0432384162305185	Test Loss:  1.0370329970933505
Train Accuracy:  98.52377300613497% 	Test Accuracy:  78.29754601226993% 

Epoch  6
-----------------
Train Loss:  0.03554717775328375	Test Loss:  0.8759748727208906
Train Accuracy:  98.71549079754601% 	Test Accuracy:  79.21779141104295% 

Epoch  7
-----

In [16]:
torch.save(classifier, "BERTTransferHumor.pth")